# 自定义数据集
- 用于构建训练的输入和标签
- 标签和输入都是同一句话，只不过标签向后移动一位
    - 输入：["The", "cat", "sat", "on", "the"]
    - 标签：["cat", "sat", "on", "the", "mat"]

In [1]:
import torch
import torch.utils.data as Data

In [2]:
# 预处理文本的方法
# 这个函数的输入输出是什么？有什么作用？
# 制:表符 \t 被替换为 <sep>
def make_data(datas):
    train_datas = []
    for data in datas:
        data = data.strip() # 去除首尾空格
        train_data = [i if i != '\t' else "<sep>" for i in data] + ['<sep>']
        train_datas.append(train_data)
    return train_datas

In [ ]:
# 自定义数据集
# 需要返回输入和标签
class MyDataSet(Data.Dataset):
    
    # 导入预处理好的训练集
    def __init__(self, datas):
        self.datas = datas
        
    # 获取输入和标签
    def __getitem__(self, idx):
        data = self.datas[idx]
        decoder_input = data[:-1] # 输入是所有文本
        decoder_output = data[1:] # 输出也就是标签是移动一位的同一个文本
        
        decoder_input_len = len(decoder_input)
        decoder_output_len = len(deocder_output)
        
        return {
            "decoder_input":decoder_input,
            "decoder_input_len":decoder_input_len,
            "decoder_output":decoder_output,
            "decoder_output_len":decoder_output_len
        }
    
    def __len__(self):
        return len(self.datas)
    
    # 这段函数是什么意思？对一个batch的数据进行填充保证一样的长度大小
    def padding_batch(self, batch):
        decoder_input_lens = [d["decoder_input_len"] for d in batch]
        decoder_output_lens = [d["decoder_output_len"] for d in batch]
        
        decoder_input_maxlen = max(decoder_input_lens)
        decoder_output_maxlen = max(decoder_output_lens)
        
        for d in batch:
            d["decoder_input"].extend([word2id["<pad>"]]*(decoder_input_maxlen-d["decoder_input_len"]))
            d["decoder_output"].extend([word2id["<pad>"]]*(decoder_output_maxlen-d["decoder_output_len"]))
        decoder_inputs = torch.tensor([d["decoder_input"] for d in batch], dtype=torch.long)
        decoder_outputs = torch.tensor([d["decoder_output"] for d in batch], dtype=torch.long)
        
        return decoder_inputs, decoder_outputs
    